In [1]:
#quick script to find the autocorrelation in time of front frequency for each 1 degree latitude bin
#without accounting for autocorrelation, changes in frequency will be found to be more significant
#than they actually are

#a bunch of this code is copied over from RDA_hov_periods.nc and slightly adapted.

In [2]:
import numpy as np
import time
import os
import netCDF4 as nc
import datetime
import matplotlib.pyplot as plt
from autocorr import autocorr
%matplotlib notebook

In [3]:
#Access NetCDF files (currently hosted in Ferret directory)
RDA_path_1 = "/Users/Siwen/Desktop/ferret/bin/meiyu_clean.nc"
RDA_path_2 = "/Users/Siwen/Desktop/ferret/bin/meiyu_2_clean.nc"
RDA_1 = nc.Dataset(RDA_path_1, 'r')
RDA_2 = nc.Dataset(RDA_path_2, 'r')

In [4]:
#load data from NetCDF files to notebook
lat_1_all =  RDA_1.variables['lat_115'][:]
lat_2_all =  RDA_2.variables['lat_115'][:]
intensity_1_all = RDA_1.variables['intensity'][:] 
intensity_2_all = RDA_2.variables['intensity'][:]

RDA_1.close()
RDA_2.close()

In [5]:
#Assign a calendar date to each time point
startday = datetime.datetime(1951,1,1)
date_list = np.array([datetime.timedelta(days=x) + startday for x in range(0, 20819)])

In [6]:
#quick function that bins everything in a 2D array and puts floor and ceiling on array values.
def findbins(lat):
    index = np.ndarray.round(lat-19.5)
    index[index < 0] = 0
    index[index > 21] = 21
    return index

In [7]:
#bin latitudes
lat_1_indices = findbins(lat_1_all)
lat_2_indices = findbins(lat_2_all)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in greater


In [8]:
#create a time series of whether a rainband exists for each latitude bin
occupancy_1 = np.zeros([20819,22])
occupancy_2 = np.zeros([20819,22])

for i, index in enumerate(lat_1_indices):
    if ~np.isnan(index):
        occupancy_1[i,index] = 1
        
for i, index in enumerate(lat_2_indices):
    if ~np.isnan(index):
        occupancy_2[i,index] = 1

#also create combined occupancy matrix of both primary and secondary fronts
occupancy = occupancy_1 + occupancy_2

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [9]:
def smooth(P,dayrange,latrange):
    Psmooth=np.zeros(P.shape)
    
    ll=int((dayrange-1)/2)
    yy=int((latrange-1)/2)
    
    days = P.shape[0]
    lats = P.shape[1]
    
    for d in range(days):
        for j in range(lats):
            
            Psample = P.take(range(d-ll,d+ll+1), mode='wrap', axis=0)
            Ps = Psample[:,max(0,j-yy):min(days,j+yy+1)]
            Psmooth[d,j] = np.mean(np.mean(Ps))
    
    return Psmooth

In [10]:
#switching it to binary - hence the *5. Wanted to keep same def of smooth function as elsewhere
#NOTE: in some cases, values can reach 2 (if both fronts are within small latitude range on a given day)
#should be a minimal effect - we test both keeping the 2 and reducing to 1.
occupancy_1_smth = smooth(occupancy_1,1,5)*5
occupancy_2_smth = smooth(occupancy_2,1,5)*5
occupancy_smth = smooth(occupancy,1,5)*5


In [11]:
#as constructed, occupancy_smth can have values of 2 if fronts are close enough - this fixes that.
occupancy_smth_binary = np.copy(occupancy_smth) #without using copy, still points to same array.
occupancy_smth_binary[occupancy_smth_binary>1] = 1

In [12]:
#quick comparison of how much the smoothing in latitude changes mean front occupancy
print(np.mean(occupancy, axis=0))
print(np.mean(occupancy_smth, axis=0))

[ 0.00048033  0.00086459  0.00384264  0.02877179  0.02656227  0.03227821
  0.03443969  0.03818627  0.04303761  0.03669725  0.03247034  0.02987656
  0.01902109  0.01998175  0.01565877  0.01508238  0.01196023  0.00979874
  0.00667659  0.00614823  0.00403478  0.00134493]
[ 0.00864595  0.04244921  0.06052164  0.09231952  0.12589462  0.16023824
  0.17450406  0.18463903  0.18483116  0.18026802  0.16110284  0.13804698
  0.1170085   0.09962054  0.08170421  0.07248187  0.05917671  0.04966617
  0.03861857  0.02800327  0.02275566  0.01921322]


In [13]:
ac_1 = np.array([ autocorr(x)[1] for x in occupancy_1.T ])
ac_2 = np.array([ autocorr(x)[1] for x in occupancy_2.T ])
ac = np.array([ autocorr(x)[1] for x in occupancy.T ])

ac_smth_1 = np.array([ autocorr(x)[1] for x in occupancy_1_smth.T ])
ac_smth_2 = np.array([ autocorr(x)[1] for x in occupancy_2_smth.T ])
ac_smth = np.array([ autocorr(x)[1] for x in occupancy_smth.T ])

ac_smth_bin = np.array([ autocorr(x)[1] for x in occupancy_smth_bin.T ])

NameError: name 'occupancy_smth_bin' is not defined

In [ ]:
print(ac_smth_bin)

In [43]:
## FIND AUTOCORRELATION FOR ARBITRARY TIME PERIODS ##
#much like the RDA_freq_diff script, now taking the next step and letting autocorrelation be calculated over arbitrary
#time periods.

#it's reasonable for autocorrelation to change a lot across the year, so worth checking.
day_beg = [1,61,121,161,201,274,121,304,321]
day_end = [365,120,160,200,273,320,303,120,60]

#stores autocorrelation time scales for each latitude box.
tau_1 = [] #primary front only
tau_2 = [] #secondary front only
tau = [] #both primary and secondary

for db, de in zip(day_beg, day_end):
    
    #date_list defined above as list of dates between January 1, 1951 and Dec 31, 2007
    #mysterious .timetuple().tm_yday function below returns day of year given date.
    if db <= de:
        mymap = map(lambda date: (date.timetuple().tm_yday >= db) & (date.timetuple().tm_yday <= de), date_list)
    else: #if de is earlier in the year than db, then we have to wrap around the new year.
        mymap = map(lambda date: not((date.timetuple().tm_yday > de) & (date.timetuple().tm_yday < db)), date_list)
        
    extract = np.array(list(mymap))
    
    ## primary fronts
    oc_1 = np.copy(occupancy_1_smth)
    oc_1_mean = np.mean(occupancy_1_smth[extract],0) #this is the mean during time period of interest
    oc_1[~extract] = oc_1_mean #change all values outside of time period of interest to mean
    #therefore, won't contribute to autocorrelation
    
    ## secondary fronts
    oc_2 = np.copy(occupancy_2_smth)
    oc_2_mean = np.mean(occupancy_2_smth[extract],0) #this is the mean during time period of interest
    oc_2[~extract] = oc_2_mean #change all values outside of time period of interest to mean
    #therefore, won't contribute to autocorrelation
    
    ## all fronts
    oc = np.copy(occupancy_smth)
    oc_mean = np.mean(occupancy_smth[extract],0) #this is the mean during time period of interest
    oc[~extract] = oc_mean #change all values outside of time period of interest to mean
    #therefore, won't contribute to autocorrelation
    
    tau_1.append(np.array([ autocorr(x)[1] for x in oc_1.T ]))
    tau_2.append(np.array([ autocorr(x)[1] for x in oc_2.T ]))
    tau.append(np.array([ autocorr(x)[1] for x in oc.T ]))
    
    print(db,de)
    #print(tau_1[-1])
    #print(tau_2[-1])
    print(tau[-1])
    
    #small debugging module to verify that the actual standard deviation of our hacked time series is the same as
    #the theoretical expectation for a Bernoulli time-series (p*(1-p))**.5
    print((oc_mean*(1-oc_mean))**.5)
    print(np.std(oc,0))    

    oc_test = (365/(de-db+1))**.5*(np.copy(oc)-oc_mean)
    print(np.std(oc_test,0))
    
    time.sleep(10)
    
    #tau_1.append()
    #tau_2.append()

1 365
[ 1.26786473  2.06815904  2.19667512  2.41851382  2.63731245  2.91657888
  3.12864151  3.14235874  3.09085791  3.01693047  2.75478669  2.69934251
  2.78417335  3.09818152  3.48510832  3.55233922  3.31952351  3.067708
  2.65380782  2.42789364  2.0907967   1.93847172]
[ 0.09261163  0.20154014  0.23843724  0.28949626  0.33177188  0.36688221
  0.37964302  0.38810541  0.38826149  0.3845114   0.36772569  0.34504673
  0.32152421  0.29958298  0.27399749  0.25936451  0.23602952  0.21732359
  0.19274635  0.16503597  0.14917256  0.13731898]
[ 0.11972933  0.22624855  0.23835706  0.28939891  0.33166031  0.36675883
  0.37951535  0.38809868  0.38850201  0.3847568   0.36812432  0.34520909
  0.32156549  0.29948223  0.27390535  0.25927729  0.23595015  0.21725051
  0.19268153  0.16498047  0.16711194  0.17791187]
[ 0.23945865  0.45249709  0.47671412  0.57879781  0.66332063  0.73351766
  0.7590307   0.77619736  0.77700403  0.7695136   0.73624864  0.69041818
  0.64313098  0.59896447  0.5478107   0.518

KeyboardInterrupt: 

In [31]:
365**.5/40**.5

3.0207614933986426

In [68]:
date_list[5000].timetuple().tm_yday

253